In [1]:
def get_input(filename, part=1):
     with open(filename, "r") as f:
          in1, in2 = f.read().split("\n\n")
          grid = {}
          for i, line in enumerate(in1.splitlines()):
               if part == 2:
                    line = (
                         line
                         .replace("#", "##")
                         .replace(".", "..")
                         .replace("O", "[]")
                         .replace("@", "@.")
                    )
               for j, s in enumerate(line):
                    grid[i, j] = s
                    if s == "@":
                         robot = (i, j)
     moves = [s for s in in2 if s != "\n"]
     return grid, moves, robot

In [2]:
def print_grid(grid):
    i_max = max(x[0] for x in grid)
    j_max = max(x[1] for x in grid)
    for i in range(i_max + 1):
        print("".join([grid[i, j] for j in range(j_max + 1)]))
        
def boxes(i, j, di, dj, grid):
    n_boxes = 0
    n = 1
    while True:
        ni, nj = i + n * di, j + n * dj
        if grid[ni, nj] == ".":
            return True, n_boxes
        if grid[ni, nj] == "#":
            return False, n_boxes
        if grid[ni, nj] == "O":
            n_boxes += 1
            n += 1

In [3]:
grid, moves, robot = get_input("15_input.txt")
dirs = {"<": (0, -1), "^": (-1, 0), ">": (0, 1), "v": (1, 0)}
i, j = robot
for move in moves:
    di, dj = dirs[move]
    can_move, n_boxes = boxes(i, j, di, dj, grid)
    if not can_move:
        continue
    grid[i + di, j + dj] = "@"
    grid[i, j] = "."
    if n_boxes > 0:
        grid[i + (n_boxes + 1) * di, j + (n_boxes + 1) * dj] = "O"
    i += di
    j += dj

gps = 0
for (i, j), s in grid.items():
    if s == "O":
        gps += i * 100 + j
print(gps)

1485257


In [4]:
def boxes_r(i, j, grid):
    if grid[i, j + 1] == ".":
        return True, []
    if grid[i, j + 1] == "#":
        return False, []
    if grid[i, j + 1] == "[":
        lr = boxes_r(i, j + 2, grid)
        return lr[0], [(i, j + 1), (i, j + 2)] + lr[1]
    
def boxes_l(i, j, grid):
    if grid[i, j - 1] == ".":
        return True, []
    if grid[i, j - 1] == "#":
        return False, []
    if grid[i, j - 1] == "]":
        lr = boxes_l(i, j - 2, grid)
        return lr[0], [(i, j - 1), (i, j - 2)] + lr[1]
        
def boxes_u(i, j, grid):
    if grid[i - 1, j] == ".":
        return True, set()
    if grid[i - 1, j] == "#":
        return False, set()
    if grid[i - 1, j] == "]":
        up1 = boxes_u(i - 1, j, grid)
        up2 = boxes_u(i - 1, j - 1, grid)
        can_move = up1[0] and up2[0]
        pos_move = set([(i - 1, j), (i - 1, j - 1)]).union(up1[1], up2[1])
        return can_move, pos_move
    elif grid[i - 1, j] == "[":
        up1 = boxes_u(i - 1, j, grid)
        up2 = boxes_u(i - 1, j + 1, grid)
        can_move = up1[0] and up2[0]
        pos_move = set([(i - 1, j), (i - 1, j + 1)]).union(up1[1], up2[1])
        return can_move, pos_move
    
def boxes_d(i, j, grid):
    if grid[i + 1, j] == ".":
        return True, set()
    if grid[i + 1, j] == "#":
        return False, set()
    if grid[i + 1, j] == "]":
        up1 = boxes_d(i + 1, j, grid)
        up2 = boxes_d(i + 1, j - 1, grid)
        can_move = up1[0] and up2[0]
        pos_move = set([(i + 1, j), (i + 1, j - 1)]).union(up1[1], up2[1])
        return can_move, pos_move
    elif grid[i + 1, j] == "[":
        up1 = boxes_d(i + 1, j, grid)
        up2 = boxes_d(i + 1, j + 1, grid)
        can_move = up1[0] and up2[0]
        pos_move = set([(i + 1, j), (i + 1, j + 1)]).union(up1[1], up2[1])
        return can_move, pos_move

In [5]:
grid, moves, robot = get_input("15_input.txt", part=2)
dirs = {"<": (0, -1), "^": (-1, 0), ">": (0, 1), "v": (1, 0)}
i, j = robot
stop = False
c = 0
for move in moves:
    if move == "<":
        can_move, pos_move = boxes_l(i, j, grid)
    elif move == ">":
        can_move, pos_move = boxes_r(i, j, grid)
    elif move == "^":
        can_move, pos_move = boxes_u(i, j, grid)
    elif move == "v":
        can_move, pos_move = boxes_d(i, j, grid)
    if not can_move:
        continue
    di, dj = dirs[move]

    if move == "<":
        if pos_move:
            to_move = sorted(list(pos_move), key=lambda x: x[1])
            for ii, jj in to_move:
                grid[ii, jj - 1] = grid[ii, jj]
            grid[to_move[-1]] = "."
    if move == ">":
        if pos_move:
            to_move = sorted(list(pos_move), key=lambda x: x[1], reverse=True)
            for ii, jj in to_move:
                grid[ii, jj + 1] = grid[ii, jj]
            grid[to_move[-1]] = "."
    if move == "^":
        if pos_move:
            for jj in set([x[1] for x in pos_move]):
                ii_min = min(x[0] for x in pos_move if x[1] == jj)
                ii_max = max(x[0] for x in pos_move if x[1] == jj)
                for ii in range(ii_min, ii_max + 1):
                    if grid[ii - 1, jj] in "[]" and(ii - 1, jj) not in pos_move:
                        continue
                    if grid[ii, jj] in "[]" and (ii, jj) not in pos_move:
                        grid[ii - 1, jj] = "."
                    else:
                        grid[ii - 1, jj] = grid[ii, jj]
                grid[ii_max, jj] = "."
    if move == "v":
        if pos_move:
            for jj in set([x[1] for x in pos_move]):
                ii_min = min(x[0] for x in pos_move if x[1] == jj)
                ii_max = max(x[0] for x in pos_move if x[1] == jj)
                for ii in range(ii_max, ii_min - 1, -1):
                    if grid[ii + 1, jj] in "[]" and (ii + 1, jj) not in pos_move:
                        continue
                    if grid[ii, jj] in "[]" and(ii, jj) not in pos_move:
                        grid[ii + 1, jj] = "."
                    else:
                        grid[ii + 1, jj] = grid[ii, jj]
                grid[ii_min, jj] = "."
    grid[i + di, j + dj] = "@"
    grid[i, j] = "."
    i += di
    j += dj
    
gps = 0
for (i, j), s in grid.items():
    if s == "[":
        gps += i * 100 + j
print(gps)

1475512
